This is the first part of item based collaborative filtering. We will be using the movielens dataset. 

In [1]:
import pandas as pd
import numpy as np
r_cols = ['user_id','movie_id','rating']
ratings = pd.read_csv('u.data', sep='\t', names = r_cols, usecols=range(3),encoding="ISO-8859-1")
ratings


,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3
...,...,...,...
99998,880,476,3
99999,716,204,5
100000,276,1090,1
100001,13,225,2


In [2]:
m_cols =['movie_id','movie_title']
movies = pd.read_csv('u.item',sep='|', names=m_cols, usecols=range(2),encoding="ISO-8859-1")
ratings = pd.merge(movies, ratings)
ratings

,movie_id,movie_title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3
...,...,...,...,...
99998,1678,Mat' i syn (1997),863,1
99999,1679,B. Monkey (1998),863,3
100000,1680,Sliding Doors (1998),863,2
100001,1681,You So Crazy (1994),896,3


In [3]:
# we want to find the relationship between movies based on each user who watch a set of movies
movieratings = ratings.pivot_table(index=['user_id'], columns=['movie_title'], values ='rating')
movieratings.head()

movie_title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#lets now get the rating of a particular movie
younggunsrating = movieratings['Young Guns (1988)']
younggunsrating.head()

user_id
0    NaN
1    3.0
2    NaN
3    NaN
4    NaN
Name: Young Guns (1988), dtype: float64

In [5]:
# pairwise correlation of younggun vector of user rating with every other movie
similarmovies = movieratings.corrwith(younggunsrating)
similarmovies =  similarmovies.dropna()
df = pd.DataFrame(similarmovies)
df.columns =['similarity']
df.head()

C:\Users\shubh\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\shubh\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,similarity
movie_title,
101 Dalmatians (1996),0.119234
12 Angry Men (1957),0.068944
187 (1997),-0.500000
2 Days in the Valley (1996),0.156220
"20,000 Leagues Under the Sea (1954)",-0.010894


In [6]:
df.sort_values(by=['similarity'], inplace=True, ascending=False)

In [7]:
df.head(100)

,similarity
movie_title,
Nothing to Lose (1994),1.000000
8 Seconds (1994),1.000000
Set It Off (1996),1.000000
Denise Calls Up (1995),1.000000
Picture Bride (1995),1.000000
...,...
Airheads (1994),0.616261
Solo (1996),0.612372
In the Company of Men (1997),0.612372


In [15]:
# here if we see the similarity to the startrek movie it doesnt make sense, so let us refine this
movieStats = ratings.groupby('movie_title').agg({'rating':['count','mean']})
movieStats

rating          
                                       count      mean
movie_title                                           
'Til There Was You (1997)                  9  2.333333
1-900 (1994)                               5  2.600000
101 Dalmatians (1996)                    109  2.908257
12 Angry Men (1957)                      125  4.344000
187 (1997)                                41  3.024390
...                                      ...       ...
Young Guns II (1990)                      44  2.772727
Young Poisoner's Handbook, The (1995)     41  3.341463
Zeus and Roxanne (1997)                    6  2.166667
unknown                                    9  3.444444
Á köldum klaka (Cold Fever) (1994)         1  3.000000

[1664 rows x 2 columns]

In [22]:
popularMovies = movieStats['rating']['count']>=100
popularMovies
movieStats[popularMovies].sort_values([('rating','mean')], ascending=False)[:15]

rating          
                                        count      mean
movie_title                                            
Close Shave, A (1995)                     112  4.491071
Schindler's List (1993)                   298  4.466443
Wrong Trousers, The (1993)                118  4.466102
Casablanca (1942)                         243  4.456790
Shawshank Redemption, The (1994)          283  4.445230
Rear Window (1954)                        209  4.387560
Usual Suspects, The (1995)                267  4.385768
Star Wars (1977)                          584  4.359589
12 Angry Men (1957)                       125  4.344000
Citizen Kane (1941)                       198  4.292929
To Kill a Mockingbird (1962)              219  4.292237
One Flew Over the Cuckoo's Nest (1975)    264  4.291667
Silence of the Lambs, The (1991)          390  4.289744
North by Northwest (1959)                 179  4.284916
Godfather, The (1972)                     413  4.283293

In [23]:
df = movieStats[popularMovies].join(pd.DataFrame(similarmovies, columns=['similarity']))
df.head(10)

C:\Users\shubh\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(rating, count)","(rating, mean)",similarity
movie_title,,,
101 Dalmatians (1996),109,2.908257,0.119234
12 Angry Men (1957),125,4.344000,0.068944
2001: A Space Odyssey (1968),259,3.969112,-0.174918
Absolute Power (1997),127,3.370079,0.254324
"Abyss, The (1989)",151,3.589404,0.384703
Ace Ventura: Pet Detective (1994),103,3.048544,0.360457
"Adventures of Priscilla, Queen of the Desert, The (1994)",111,3.594595,-0.027420
"African Queen, The (1951)",152,4.184211,0.042534
Air Force One (1997),431,3.631090,0.349531


In [24]:
df.sort_values(['similarity'], ascending=False)[:15]

,"(rating, count)","(rating, mean)",similarity
movie_title,,,
Young Guns (1988),101,3.207921,1.000000
Good Will Hunting (1997),198,4.262626,0.675082
"River Wild, The (1994)",146,3.143836,0.662424
"Frighteners, The (1996)",115,3.234783,0.629871
Seven Years in Tibet (1997),155,3.458065,0.599263
"Time to Kill, A (1996)",232,3.685345,0.597969
"Ghost and the Darkness, The (1996)",128,3.203125,0.585139
"Nightmare on Elm Street, A (1984)",111,3.171171,0.578651
"First Wives Club, The (1996)",160,3.018750,0.569725
